In [159]:
import os
import pandas as pd

In [160]:
def get_game_codes():

    directory = './demo_data/'

    files = os.listdir(directory)

    game_codes = []

    for file in files:
        game_code = file.split('_')[0]
        game_codes.append(game_code)

    game_codes = list(set(game_codes))

    return game_codes

In [161]:
def get_data(game_code, event_type):

    directory = './demo_data/'

    filename = f'{directory}{game_code}_{event_type}.txt'

    df = pd.read_csv(filename, sep='\t')

    return df

In [162]:
def save_df_translated(game_code, event_type, df):

    directory = './data_translated/'

    filename = f'{directory}{game_code}_{event_type}.csv'

    df.to_csv(filename)

In [163]:
def translate_meta(game_code, event_type):

    df = get_data(game_code, event_type)

    # English translations
    english_translations = {
    'codigo': 'code',
    'tipo': 'type',
    'equipo': 'team',
    'player': 'player',
    'x': 'x',
    'y': 'y',
    'min': 'minute',
    'sec': 'second',
    'period': 'period',
    'partido': 'match',
    'nombre_equipo': 'team_name',
    'nombre_jugador': 'player_name',
    'outcome': 'outcome',
    'fase': 'phase',
    'end_x': 'end_x',
    'end_y': 'end_y',
    'extra': 'extra'
    }

    # Renaming the columns based on the English translations
    df.rename(columns=english_translations, inplace=True)

    return df

In [164]:
def translate_type(row):

    translations = {
        'pase': 'pass',
        'pase_fallado': 'failed_pass',
        'balon_fuera': 'ball_out',
        'recuperacion_balon': 'ball_recovery',
        'falta_recibida': 'foul_received',
        'falta_cometida': 'foul_committed',
        'bloqueo_pase': 'pass_block',
        'regate_recibido': 'dribble_received',
        'regate_conseguido': 'dribble_successful',
        'toco_balon': 'touched_ball',
        'duelo_aereo': 'aerial_duel',
        'ocasion_fuera': 'chance_missed',
        'despeje': 'clearance',
        'corner': 'corner_kick',
        'portero_agarrra_pelota': 'goalkeeper_catches_ball',
        'regate_fallado': 'failed_dribble',
        'entrada': 'tackle',
        'sin_identificar': 'unidentified',
        'pase_cortado': 'cut_pass',
        'pase_fuera_juego': 'offside_pass',
        'me_roban_balon': 'ball_stolen',
        'gol': 'goal',
        'parada': 'save',
        'disparo_parado': 'shot_blocked',
        'tarjeta': 'card',
        'portero_sale_despeja': 'goalkeeper_clears_ball',
        'cambio_jugador_fuera': 'substitution_out',
        'cambio_jugador_dentro': 'substitution_in',
        'perdida_golpeo_rival': 'loss_due_to_opponent_strike'
    }
    
    row['type'] = translations.get(row['type'], row['type'])

    return row['type']

In [165]:
def translate_types(game_code, event_type, df):

    df['type'] = df.apply(lambda row: translate_type(row), axis=1)

    return df

In [189]:
def translate_files(game_code, event_type='all_events'):

    df = translate_meta(game_code, event_type)

    if event_type in ['all_events','key_event']:
        df = translate_types(game_code, event_type, df)

    save_df_translated(game_code, event_type, df)

In [170]:
def get_data_translated(game_code, event_type='all_events'):

    get_data(game_code, event_type)

    translate_files(game_code, event_type)
    
    directory = './data_translated/'

    filename = f'{directory}{game_code}_{event_type}.csv'

    df = pd.read_csv(filename)

    return df

In [190]:
game_codes = get_game_codes()

events = get_data_translated(game_codes[0], event_type='key_event')

In [191]:
events.type.unique()

array(['0ball_out', '0aerial_duel_lost', '0blocked_pass', '0mal_control',
       'miss', '0despeje_clearance', '0interception', '0dispossessed',
       'goal', 'attempt_saved', 'player_off', 'player_in',
       '0me_cortan_balon_cuerpo'], dtype=object)

In [17]:
from config import PitchMeta
import numpy as np
import itertools

pitch_graph = {i: [0, 0, np.zeros(shape=[PitchMeta.x * PitchMeta.y])] for i in range(PitchMeta.x * PitchMeta.y)}
pitch_graph[2][2][0]


0.0

In [119]:
from data_type import PlayerCoordinate

def get_the_closest_tile(player_coordinate: PlayerCoordinate):
    # the pitch size is 100 * 100, and converted to 16 * 12
    tile_x = round(player_coordinate.x / (100 / PitchMeta.x))
    tile_y = round(player_coordinate.y / (100 / PitchMeta.y))
    if tile_x == 16:
        tile_x = 15
    if tile_y == 12:
        tile_y = 11

    if tile_y == 0:
        tile_id = tile_x
    else:
        tile_id =  tile_y * PitchMeta.x + tile_x
    
    return tile_x, tile_y, tile_id
    #return tile_id

In [120]:
# Test the four corners of the pitch
corners = [
    PlayerCoordinate(0, 0),
    PlayerCoordinate(100, 0),
    PlayerCoordinate(0, 100),
    PlayerCoordinate(100, 100),
    PlayerCoordinate(50, 50)
]

for corner in corners:
    tile_x, tile_y, tile_id = get_the_closest_tile(corner)
    print(f"Player Coordinate: ({corner.x}, {corner.y})")
    print(f"Tile Coordinates: ({tile_x}, {tile_y})")
    print(f"Tile ID: {tile_id}")
    print("------")

Player Coordinate: (0, 0)
Tile Coordinates: (0, 0)
Tile ID: 0
------
Player Coordinate: (100, 0)
Tile Coordinates: (15, 0)
Tile ID: 15
------
Player Coordinate: (0, 100)
Tile Coordinates: (0, 11)
Tile ID: 176
------
Player Coordinate: (100, 100)
Tile Coordinates: (15, 11)
Tile ID: 191
------
Player Coordinate: (50, 50)
Tile Coordinates: (8, 6)
Tile ID: 104
------


In [121]:
int(16.6)

16